### Parse RT130 9/* files to reconstruct digitizer-station history

In [ ]:
summary9file = '/media/sda1/summary9file.csv'

In [ ]:
STATIONS = ['BHP', 'TANK', 'FIRE', 'BCHH', 'DVEL', 'RBLAB']
dayfullpaths = sorted(glob.glob('%s/20?????' % RAWDIR))
if os.path.exists(summary9file):
    os.remove(summary9file)
count = 0
filehandle = open(summary9file, "w")
for thisdayfullpath in dayfullpaths:
    count = count + 1
    print('Processing %s (%d of %d)' % (thisdayfullpath, count, len(dayfullpaths) ), end="\r", flush=True)
    thisdaydir = os.path.basename(thisdayfullpath) # a directory like 2018365
    rt130files = sorted(glob.glob('%s/????/9/*' % thisdayfullpath))
    if rt130files:
        
        for rt130file in rt130files:
            output = os.popen('strings %s' % rt130file).read()
            if output: 
                for station in STATIONS:
                    firstindex = output.find(station)
                    if firstindex > -1:
                        break
                    
                if firstindex != -1:
                    pathparts = rt130file.split('/')
                    rt130 = pathparts[-3]
                    filehandle.write('%s, %s, %s\n' % (output[firstindex:firstindex+4], thisdaydir, rt130) )
         
        #if count>40:
        #    break
        
filehandle.close()
print('\nDone')

In [ ]:
output = os.popen('sort %s | uniq' % summary9file).read()
#print(output)
lastStationDigitizerCombo = ""
lastline = ""

for thisline in output.split('\n'):
    try:
        (station, yyyyjjj, digitizer)  = thisline.split(',')
        thisStationDigitizerCombo = '%s%s' % (station, digitizer)
        #print(lastStationDigitizerCombo, thisStationDigitizerCombo)
        if lastStationDigitizerCombo != thisStationDigitizerCombo:
            print(lastline)
            print(thisline)
        lastStationDigitizerCombo = thisStationDigitizerCombo
        lastline = thisline
    except:
        pass
print(lastline)

### Parse RT130 1/* files to reconstruct digitizer-GPS_Position history
This generates a DataFrame/CSV file like:

<pre>
,Digitizer,yyyyjjj,Latitude,LatSTD
0,92B7,2018222,28.0595,2.8867513458681453e-05
1,92B7,2018223,28.0595,1.8559214542252517e-05
2,92B7,2018229,28.573463888888888,2.721655270429592e-06
3,92B7,2018230,28.573469444444445,1.7899429988238652e-06
4,92B7,2018231,28.573469444444445,2.6032870393506316e-06
5,92B7,2018233,28.573469444444445,3.552713678800501e-15
6,92B7,2018234,28.573469444444445,3.552713678800501e-15
7,92B7,2018235,28.573469444444445,3.552713678800501e-15
</pre>

The next task will be to detect dates on which a digitizer was moved by examining plots of GPS Latitude & standard deviation.


In [ ]:
def find_all(a_str, sub):
    start = 0
    while True:
        start = a_str.find(sub, start)
        if start == -1: return
        yield start
        start += len(sub) # use start += 1 to find overlapping matches

#list(find_all('spam spam spam spam', 'spam'))

positionDF = pd.DataFrame(columns = ['Digitizer', 'yyyyjjj', 'Latitude', 'LatSTD'])
digitizerList = list()
yyyyjjjList = list()
latitudeList = list()
latSTDList = list()
dayfullpaths = sorted(glob.glob('%s/20?????' % RAWDIR))
summary1file = '/media/sda1/summary1file.csv'
if os.path.exists(summary1file):
    os.remove(summary1file)
count = 0
#filehandle = open(summary1file, "w")
for thisdayfullpath in dayfullpaths:
    count = count + 1
    #print('Processing %s (%d of %d)' % (thisdayfullpath, count, len(dayfullpaths) ), end="\r", flush=True)
    thisdaydir = os.path.basename(thisdayfullpath) # a directory like 2018365
    digitizerpaths = sorted(glob.glob('%s/????' % thisdayfullpath))
    for digitizerpath in digitizerpaths:
        thisdigitizer = os.path.basename(digitizerpath)
        rt130files = sorted(glob.glob('%s/1/*' % digitizerpath))
        if rt130files:
            all_positions = list()
            for rt130file in rt130files:
                print('Processing %s' % rt130file, end="\r", flush=True)
                
                output = os.popen('strings %s' % rt130file).read()
                if output: 
                    
                    all_indexes = list(find_all(output, 'POSITION'))
                
                    for each_index in all_indexes:
                        
                        Ndeg = output[each_index+11:each_index+13]
                        Nmin = output[each_index+14:each_index+16]
                        Nsec = output[each_index+17:each_index+22]
                        Ndecdeg = float(Ndeg) + float(Nmin)/60 + float(Nsec)/3600
                        
                        if Ndecdeg > 28.0 and Ndecdeg < 29.0:
                            #print(output[each_index+11:each_index+22], " ", Ndecdeg)
                            all_positions.append(Ndecdeg)
            if all_positions:  
                all_positions_np = np.array(all_positions)
                #print(all_positions_np)
                #print(np.median(all_positions_np))
                #print('%s, %s, %f, %f\n' % (thisdigitizer, thisdaydir, np.median(all_positions_np), np.std(all_positions_np)) )
                #filehandle.write('%s, %s, %f\n' % (thisdigitizer, thisdaydir, np.median(all_positions_np)  ) )
                digitizerList.append(thisdigitizer)
                yyyyjjjList.append(thisdaydir)
                latitudeList.append(np.median(all_positions_np))
                latSTDList.append(np.std(all_positions_np))
         
    #if count>20:
    #    break

        
#filehandle.close()
positionDF['Digitizer'] = digitizerList
positionDF['yyyyjjj'] = yyyyjjjList
positionDF['Latitude'] = latitudeList
positionDF['LatSTD'] = latSTDList

positionDF.to_csv(summary1file)

print('\nDone')

#### Convert yyyyjjj strings to datetime objects

In [ ]:
import datetime
#print(positionDF)
dates = list()
for thisyyyyjjj in positionDF['yyyyjjj']:
    thisdate = datetime.datetime.strptime(thisyyyyjjj, "%Y%j").date()
    dates.append(thisdate)
#print(dates)
positionDF['dates']=dates

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams['axes.formatter.useoffset'] = False # do not allow relative y-labels

allDigitizers = list(set(digitizerList))
for digitizer in allDigitizers:
    subsetDF = positionDF[positionDF['Digitizer']==digitizer]
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.plot_date(subsetDF['dates'], subsetDF['Latitude'],'*')  
    subsetDF.to_csv('latitude_' + digitizer + '.csv')
    ax.set_title(digitizer)
    if not (digitizer == 'AB13' or digitizer == '9D7C'):
        ax.set_ylim(28.572, 28.575)
    #xt = ax.get_xticks()
    #ax.set_xticks(xt[0::30])
    for xtl in ax.get_xticklabels():
        xtl.set_rotation(30)
        xtl.set_horizontalalignment('right')
    #plt.tight_layout()

In [ ]:
import matplotlib.pyplot as plt
allDigitizers = list(set(digitizerList))
fig = plt.figure()
ax = fig.add_subplot(111)
for digitizer in allDigitizers:
    subsetDF = positionDF[positionDF['Digitizer']==digitizer]
    ax.plot_date(subsetDF['dates'], subsetDF['Latitude'],'.', label=digitizer)  
    
#xt = ax.get_xticks()
#ax.set_xticks()
ax.set_xlim(dates[0],dates[-1])
for xtl in ax.get_xticklabels():
    xtl.set_rotation(30)
    xtl.set_horizontalalignment('right')
plt.legend()
ax.set_ylim(28.51, 28.58)
plt.savefig('digitizer_lats.png')
ax.set_ylim(28.572, 28.575)
plt.savefig('digitizer_lats_zoomed.png')
#plt.tight_layout()